In [1]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
data_path = "data.json"
learning_rate = 0.0001
EPOCHS = 40
BATCH_SIZE = 32
saved_model_path = "model.h5"
num_keywords = 31

In [2]:
def load_data(d_path):
    with open(d_path, "r") as fp:
        data = json.load(fp)
    x = np.array(data["MFCCs"])
    y =np.array(data["labels"])
    
    return x, y

def get_data(d_path, test_size = 0.1, validation_size = 0.1):
    x, y = load_data(d_path)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = validation_size)
    
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    x_validation = x_validation[..., np.newaxis]
    
    return x_train, x_validation, x_test, y_train, y_validation, y_test

In [3]:
def build_model(input_shape, l_rate, error = "sparse_categorical_crossentropy"):
    model = tf.keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(64, (3,3), activation = "relu",
                                 input_shape = input_shape,
                                 kernel_regularizer = keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((3,3), strides = (2,2), padding = "same"))
    
    
    model.add(keras.layers.Conv2D(32, (3,3), activation = "relu",
                                 kernel_regularizer = keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((3,3), strides = (2,2), padding = "same"))
    
    
    
    model.add(keras.layers.Conv2D(16, (2,2), activation = "relu",
                                 kernel_regularizer = tf.keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((2,2), strides = (2,2), padding = "same"))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation = "relu"))
    model.add(keras.layers.Dropout(0.3))
    
    
    model.add(keras.layers.Dense(num_keywords, activation = "softmax"))
    
    optimizer = keras.optimizers.Adam(learning_rate = l_rate)
    model.compile(optimizer= optimizer, loss = error, metrics=["accuracy"])
    
    model.summary()
    
    return model

In [4]:

def main():
    x_train, x_validation, x_test, y_train, y_validation, y_test = get_data(data_path)
    
    input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
    
    model = build_model(input_shape, learning_rate)
    
    model.fit(x_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data = (x_validation, y_validation))
    
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print("\nTest loss: {}, test accuracy: {}".format(test_loss, 100*test_acc))
    
    model.save(saved_model_path)

In [5]:
if __name__ == "__main__":
    main()
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 42, 11, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 42, 11, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 6, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 4, 32)         18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 4, 32)         128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 2, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 1, 16)          2


Test loss: 0.4753574028897646, test accuracy: 87.1095061302185
